In [20]:
import pandas as pd
import pickle
from get_words_by_Natasha import get_words
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from statistics import mean

In [7]:
from freq_of_symbol import freq_of_symbol
from freq_of_word import freq_of_word
from number_of_unique_words import number_of_unique_words
from number_of_words_that_occur_once import number_of_words_that_occur_once


In [8]:
def dataset_to_pandas_table(dataset = pickle.load(open("dataset.pkl", "rb"))):
    authors = list(set(dataset['authors']))
    dict_authors = dict()
    reverse_dict_authors = dict()
    for index in range(len(authors)):
        dict_authors[authors[index]] = index
        reverse_dict_authors[index] = authors[index]

    target = []
    for author in dataset["authors"]:
        target += [dict_authors[author]]
    feature1 = []
    feature2 = []
    for book in dataset['books']:
        words = get_words(book.lower())
        #сюда лемматизацию впихуем

        #сюда по очереди все функции
        feature1.append(freq_of_word(words, 'и'))
    dataset_for_ml = {"feature1": feature1, "targets": target}
    df = pd.DataFrame(dataset_for_ml)
    return df

In [9]:
df = dataset_to_pandas_table()

In [22]:
def validation(df, model):
    accuracy = []
    for shift in range(0, 10, 2):
        authors_seen = []
        test_ixs = []
        for ixs in range(len(df)):
            aut = df['targets'][ixs]
            if authors_seen.count(aut) < 2:
                authors_seen += [aut]
                test_ixs += [ixs + shift]
        train_ixs = list(set(range(len(df))) - set(test_ixs))
        y_test = df.iloc[test_ixs]['targets']
        x_test = df.iloc[test_ixs].drop(columns=["targets"])
        y_train = df.iloc[train_ixs]['targets']
        x_train = df.iloc[train_ixs].drop(columns=["targets"])

        rfc = model
        rfc.fit(x_train, y_train)
        accuracy.append(accuracy_score(y_test, rfc.predict(x_test)))
    return accuracy

print( mean(validation(df, ExtraTreesClassifier())))

0.072


In [23]:
from catboost import CatBoostClassifier

validation(df, CatBoostClassifier())

Learning rate set to 0.075665
0:	learn: 3.8879594	total: 201ms	remaining: 3m 21s
1:	learn: 3.8615760	total: 237ms	remaining: 1m 58s
2:	learn: 3.8366960	total: 283ms	remaining: 1m 34s
3:	learn: 3.8132866	total: 342ms	remaining: 1m 25s
4:	learn: 3.7870154	total: 400ms	remaining: 1m 19s
5:	learn: 3.7690639	total: 450ms	remaining: 1m 14s
6:	learn: 3.7494365	total: 503ms	remaining: 1m 11s
7:	learn: 3.7296604	total: 585ms	remaining: 1m 12s
8:	learn: 3.7080454	total: 663ms	remaining: 1m 12s
9:	learn: 3.6915497	total: 692ms	remaining: 1m 8s
10:	learn: 3.6721230	total: 736ms	remaining: 1m 6s
11:	learn: 3.6535882	total: 779ms	remaining: 1m 4s
12:	learn: 3.6367937	total: 837ms	remaining: 1m 3s
13:	learn: 3.6185882	total: 879ms	remaining: 1m 1s
14:	learn: 3.6023151	total: 927ms	remaining: 1m
15:	learn: 3.5854362	total: 988ms	remaining: 1m
16:	learn: 3.5705230	total: 1.03s	remaining: 59.8s
17:	learn: 3.5582975	total: 1.09s	remaining: 59.7s
18:	learn: 3.5460851	total: 1.14s	remaining: 58.8s
19:	lear

[0.08, 0.09, 0.04, 0.07, 0.07]